In [1]:
import pywt
import numpy as np
import cv2
import os
import warnings
import time
import csv

warnings.filterwarnings("ignore")

In [7]:
output_folder = "dwt"
input_folder = 'org_ecg_10sec_resize'
os.makedirs(output_folder, exist_ok=True)

In [3]:
def calculate_image_simmilarity(imageA, imageB, dontprint=False):
    # Convert images to float for accurate difference calculation
    imageA = imageA.astype(np.float32)
    imageB = imageB.astype(np.float32)
    
    # Compute absolute pixel-wise difference
    diff = abs(imageA - imageB)
    
    # Calculate the mean of all pixel differences (error)
    err = diff.mean()

     # Optionally print the error
    if not dontprint:
        print(f"Mean Pixel Error: {err}")
    return err

def get_threshold(im, compression_level):
    # base on the threshold lot of the ppercentage of the wavelet coefficient will be kept then other will set to 0
    thresh = np.sort(abs((im.ravel())))[int(im.size*(1-compression_level))]
    return thresh

# use pre compute that wavelet coefficient are stored and sliced for a particular image shape and decomposition level
def generate_wavlet_slices(x_shape, y_shape, level): 
    # Generate a dummy image of given shape
    sample_image = np.random.normal(size=(x_shape, y_shape))
    # Perform 2D wavelet decomposition to get coefficients
    sample_coeff = pywt.wavedec2(sample_image, 'db2', mode='periodization', level=level)
     # Convert coefficients to a single array and get slicing instructions
    slices_instructions = pywt.coeffs_to_array(sample_coeff)[1]
    return slices_instructions

def compress_ecg_image(image, level, compression_threshold):
    #apply wavelet decomposition to the image
    coeffs = pywt.wavedec2(image, 'db2', mode='periodization', level=level)
    
    #convert coefficients to array form for easy manipulation
    arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    
    # Determine value below which coefficients will be zeroed
    threshold = get_threshold(arr, compression_threshold)
    
    # Zero out small coefficients (thresholding)
    arr_compressed = arr * (abs(arr) > threshold)
    
    return arr_compressed, coeff_slices

def uncompress_ecg_image(compressed_arr, coeff_slices, level, im_shape):
    # Convert the compressed array back to wavelet coefficient format
    coeffs_compressed = pywt.array_to_coeffs(compressed_arr, coeff_slices, output_format='wavedecn')

    # Reconstruct the image using inverse wavelet transform
    reconstructed_image = pywt.waverecn(coeffs_compressed, 'db2', mode='periodization')
    
    # Clip values to valid range and convert to uint8(8bit)
    reconstructed_image = np.clip(reconstructed_image, 0, 255).astype(np.uint8)
    return reconstructed_image

def calculate_compression_ratio(original_path, compressed_path): # count cr
    original_size = os.path.getsize(original_path)
    compressed_size = os.path.getsize(compressed_path)
    if compressed_size == 0:
        return float('inf')
    return original_size / compressed_size

In [4]:
# Updated compression parameters
wavelet_level = 6  # Safer decomposition level
compression_ratio = 1.0  # Keep only 20% of coefficients
wavelet_type = 'db4'

In [11]:
image_paths = [os.path.join(input_folder, fname) 
               for fname in os.listdir(input_folder) 
               if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]


durations = []

In [12]:
for image_path in image_paths:
    filename = os.path.basename(image_path)
    output_image_path = os.path.join(output_folder, filename)
    input_image_path = image_path

    # Load image
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    start_time = time.time()

    # Dynamic level adjustment
    max_possible_level = pywt.dwt_max_level(min(img.shape), wavelet_type)
    level = min(wavelet_level, max_possible_level)

    # Compression pipeline
    compressed_img, slices = compress_ecg_image(img, level, compression_ratio)
    reconstructed_img = uncompress_ecg_image(compressed_img, slices, level, img.shape)

    # Save with maximum PNG compression
    cv2.imwrite(output_image_path, reconstructed_img, [cv2.IMWRITE_PNG_COMPRESSION, 9])

    # Calculate and display results
    cr = calculate_compression_ratio(input_image_path, output_image_path)
    print(f"✅ Compressed {input_image_path} -> {output_image_path} | CR: {cr:.2f}")

    end_time = time.time()
    duration = end_time - start_time
    durations.append((filename, duration))

# === Save durations to CSV ===
csv_path = 'compression_durations.csv'
with open(csv_path, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['Image Name', 'Duration (seconds)'])
    writer.writerows(durations)

print(f"✅ Compression complete. Duration saved to: {csv_path}")

✅ Compressed org_ecg_10sec_resize\ecg_record_100.png -> dwt\ecg_record_100.png | CR: 2.12
✅ Compressed org_ecg_10sec_resize\ecg_record_101.png -> dwt\ecg_record_101.png | CR: 2.14
✅ Compressed org_ecg_10sec_resize\ecg_record_102.png -> dwt\ecg_record_102.png | CR: 2.14
✅ Compressed org_ecg_10sec_resize\ecg_record_103.png -> dwt\ecg_record_103.png | CR: 2.10
✅ Compressed org_ecg_10sec_resize\ecg_record_104.png -> dwt\ecg_record_104.png | CR: 2.11
✅ Compressed org_ecg_10sec_resize\ecg_record_105.png -> dwt\ecg_record_105.png | CR: 1.96
✅ Compressed org_ecg_10sec_resize\ecg_record_106.png -> dwt\ecg_record_106.png | CR: 2.07
✅ Compressed org_ecg_10sec_resize\ecg_record_107.png -> dwt\ecg_record_107.png | CR: 1.96
✅ Compressed org_ecg_10sec_resize\ecg_record_108.png -> dwt\ecg_record_108.png | CR: 1.99
✅ Compressed org_ecg_10sec_resize\ecg_record_109.png -> dwt\ecg_record_109.png | CR: 2.04
✅ Compressed org_ecg_10sec_resize\ecg_record_111.png -> dwt\ecg_record_111.png | CR: 1.98
✅ Compress